# 00 — CRSP Data Download (S&P 500 Top 25)

This notebook connects to WRDS, selects the top 25 S&P 500 stocks by market capitalization at a target date, and downloads daily returns over a window covering the last 1,276 trading days. Data and metadata are saved to `data/` for reuse in other notebooks without re-querying WRDS.

## Prerequisites

You need a `.env` file in the project root with your WRDS credentials:
```
WRDS_USERNAME=your_wrds_username
WRDS_PASSWORD=your_wrds_password
```

Copy `.env.example` to `.env` and fill in your credentials. The `.env` file is gitignored for security.

## Saved outputs

- `data/daily_data_top25.csv` — Daily data (permno, date, ret, dlret, comnam, ticker)
- `data/permno_to_name.json` — Mapping PERMNO → Company name
- `data/permno_to_ticker.json` — Mapping PERMNO → Ticker
- `data/final_dates.json` — List of dates (ISO) in the final window
- `data/download_metadata.json` — Parameters used and result summary

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import wrds
from datetime import datetime
from dotenv import load_dotenv

# Load environment variables from .env file (look in parent directory)
load_dotenv(dotenv_path='../.env')

# =============================================================================
# PARAMETERS
# =============================================================================
end_date = '2025-01-30'              # Desired calendar date (will be adjusted to last trading day)
trading_days = 1276                  # ~ 5 years of trading days
num_stocks = 25                      # Universe size
start_date_approx = (pd.to_datetime(end_date) - pd.to_timedelta(trading_days * 1.8, unit='d')).strftime('%Y-%m-%d')

print('🚀 Connecting to WRDS and setting parameters…')

# Get WRDS credentials from environment variables
wrds_username = os.getenv('WRDS_USERNAME')
wrds_password = os.getenv('WRDS_PASSWORD')

if not wrds_username:
    raise ValueError("WRDS_USERNAME not found in .env file. Please add it.")

# Connect to WRDS with credentials
if wrds_password:
    db = wrds.Connection(wrds_username=wrds_username, wrds_password=wrds_password)
else:
    # If no password is provided, WRDS will prompt or use pgpass
    db = wrds.Connection(wrds_username=wrds_username)

print('✅ Connected to WRDS.')

# =============================================================================
# STEP 1: FIND LAST TRADING DAY <= end_date
# =============================================================================
query_last_date = f"SELECT max(date) as last_date FROM crsp.dsf WHERE date <= '{end_date}'"
last_date_df = db.raw_sql(query_last_date, date_cols=['last_date'])
last_trading_day_str = last_date_df['last_date'][0].strftime('%Y-%m-%d')
end_date = last_trading_day_str
print(f'📅 Last trading day selected: {end_date}')

# =============================================================================
# STEP 2: BUILD UNIVERSE (TOP 25 S&P 500 by market cap)
# =============================================================================
print('\n🚀 Selecting universe (Top 25 S&P 500)…')
query_universe = f"""
    WITH sp500_constituents AS (
        SELECT permno FROM crsp.msp500list WHERE '{end_date}' BETWEEN start AND ending
    ),
    market_cap AS (
        SELECT a.permno, ABS(a.prc * a.shrout) as mktcap
        FROM crsp.dsf AS a
        JOIN sp500_constituents AS b ON a.permno = b.permno
        WHERE a.date = '{end_date}' AND a.prc IS NOT NULL AND a.shrout IS NOT NULL
    )
    SELECT permno FROM market_cap ORDER BY mktcap DESC LIMIT {num_stocks}
"""
top_25_permno = db.raw_sql(query_universe)['permno'].tolist()
permno_tuple = tuple(top_25_permno)
print(f'✅ Universe identified: {len(top_25_permno)} stocks.')

# =============================================================================
# STEP 3: DOWNLOAD HISTORICAL DATA (RET, DLRET, + LABELS)
# =============================================================================
print('\n🚀 Downloading daily data (RET, DLRET, COMNAM, TICKER)…')
query_data = f"""
    SELECT a.permno, a.date, a.ret, b.dlret, c.comnam, c.ticker
    FROM crsp.dsf AS a
    LEFT JOIN crsp.dsedelist AS b
        ON a.permno = b.permno AND a.date = b.dlstdt
    LEFT JOIN crsp.msenames AS c
        ON a.permno = c.permno AND a.date BETWEEN c.namedt AND c.nameendt
    WHERE a.permno IN {permno_tuple} AND a.date BETWEEN '{start_date_approx}' AND '{end_date}'
"""
daily_data = db.raw_sql(query_data, date_cols=['date'])

# Fill missing comnam/ticker labels by PERMNO (ffill/bfill to cover gaps)
daily_data['comnam'] = daily_data.groupby('permno')['comnam'].transform(lambda x: x.ffill().bfill())
daily_data['ticker'] = daily_data.groupby('permno')['ticker'].transform(lambda x: x.ffill().bfill())

# Create useful mappings (convert numpy int64 keys to regular Python int for JSON compatibility)
permno_to_name = {int(k): v for k, v in daily_data.drop_duplicates('permno').set_index('permno')['comnam'].to_dict().items()}
permno_to_ticker = {int(k): v for k, v in daily_data.drop_duplicates('permno').set_index('permno')['ticker'].to_dict().items()}

# Limit to the last 1,276 trading days actually present
all_dates = sorted(daily_data['date'].unique())
final_dates = all_dates[-trading_days:] if len(all_dates) >= trading_days else all_dates
daily_data = daily_data[daily_data['date'].isin(final_dates)]

print('✅ Download completed.')

# =============================================================================
# SAVE RESULTS TO ../data/ (parent directory)
# =============================================================================
print('\n💾 Saving outputs to ../data/ …')
data_dir = os.path.join('..', 'data')
os.makedirs(data_dir, exist_ok=True)

daily_csv_path = os.path.join(data_dir, 'daily_data_top25.csv')
permno_to_name_path = os.path.join(data_dir, 'permno_to_name.json')
permno_to_ticker_path = os.path.join(data_dir, 'permno_to_ticker.json')
final_dates_path = os.path.join(data_dir, 'final_dates.json')
meta_path = os.path.join(data_dir, 'download_metadata.json')

# Convert dates to ISO for JSON
final_dates_iso = [pd.to_datetime(d).strftime('%Y-%m-%d') for d in final_dates]

# Save files
daily_data.to_csv(daily_csv_path, index=False)
with open(permno_to_name_path, 'w', encoding='utf-8') as f: 
    json.dump(permno_to_name, f, ensure_ascii=False, indent=2)
with open(permno_to_ticker_path, 'w', encoding='utf-8') as f: 
    json.dump(permno_to_ticker, f, ensure_ascii=False, indent=2)
with open(final_dates_path, 'w', encoding='utf-8') as f: 
    json.dump(final_dates_iso, f, ensure_ascii=False, indent=2)

metadata = {
    'generated_at': datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ'),
    'parameters': {
        'requested_end_date': end_date,
        'trading_days_target': trading_days,
        'num_stocks': num_stocks,
        'start_date_approx': start_date_approx
    },
    'universe': {
        'count': len(top_25_permno),
        'permnos': top_25_permno
    },
    'date_window': {
        'start': final_dates_iso[0] if final_dates_iso else None,
        'end': final_dates_iso[-1] if final_dates_iso else None,
        'n_trading_days': len(final_dates_iso)
    },
    'files': {
        'daily_csv': daily_csv_path,
        'permno_to_name': permno_to_name_path,
        'permno_to_ticker': permno_to_ticker_path,
        'final_dates': final_dates_path
    }
}
with open(meta_path, 'w', encoding='utf-8') as f: 
    json.dump(metadata, f, ensure_ascii=False, indent=2)

print('📁 Files written:')
print(' -', daily_csv_path)
print(' -', permno_to_name_path)
print(' -', permno_to_ticker_path)
print(' -', final_dates_path)
print(' -', meta_path)

# =============================================================================
# CLOSE WRDS CONNECTION
# =============================================================================
db.close()
print('🔒 WRDS connection closed.')

print('\n🎉 Done. You can now load these files from ../data/ in your other notebooks.')

🚀 Connecting to WRDS and setting parameters…
Loading library list...
Loading library list...
Done
✅ Connected to WRDS.
📅 Last trading day selected: 2024-12-31

🚀 Selecting universe (Top 25 S&P 500)…
Done
✅ Connected to WRDS.
📅 Last trading day selected: 2024-12-31

🚀 Selecting universe (Top 25 S&P 500)…
✅ Universe identified: 25 stocks.

🚀 Downloading daily data (RET, DLRET, COMNAM, TICKER)…
✅ Universe identified: 25 stocks.

🚀 Downloading daily data (RET, DLRET, COMNAM, TICKER)…
✅ Download completed.

💾 Saving outputs to ../data/ …
✅ Download completed.

💾 Saving outputs to ../data/ …


TypeError: keys must be str, int, float, bool or None, not int64